# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI 'gpt-3.5-turbo' will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Positive'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Makes decision on which items to include in menu.'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [5]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [6]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [7]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                llm = llm)

In [8]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Makes decisions on which items to include in Menu'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 1),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([dish_price])
    ]

In [9]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [10]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: Assigned Task is to generate 5 menu items based on Italian food choices with name, description, ingredients, and price. No subtasks have been completed yet.
Thoughts: To complete the Assigned Task, the menu items need to be created with the required details. The Equipped Functions available can be utilized to generate the necessary components for each menu item.
Subtask identified: Generate interesting dish names and descriptions for the 5 menu items based on Italian food choices.
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for the 5 menu items based on Italian food choices.'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the Assigned Task.
Thoughts: To complete the Assigned Task, I will need to generate interesting dish names and descriptions for the 5 menu items based on Italian food choices. Since no specific subtasks have been completed yet, I wi

In [11]:
output = my_agent.reply_user()

As the agent named Menu Creator, I have successfully completed the task of generating 5 menu items based on Italian food choices. The menu items are as follows:

1. Bella Vita Pasta
Description: A delightful combination of al dente spaghetti tossed in a rich tomato and basil sauce, topped with freshly grated Parmesan cheese. The perfect representation of the Italian zest for life.
Ingredients: spaghetti, tomato, basil, Parmesan cheese
Price: $12.99

2. Tuscan Sunset Risotto
Description: Creamy Arborio rice cooked to perfection with saffron, sun-dried tomatoes, and a hint of garlic. This dish captures the essence of a picturesque Tuscan sunset in every bite.
Ingredients: Arborio rice, saffron, sun-dried tomatoes, garlic
Price: $14.99

3. Amore Pizza Bianca
Description: A thin crust pizza topped with creamy ricotta, mozzarella, and a drizzle of truffle oil. Each bite is a love letter to the simplicity and elegance of Italian cuisine.
Ingredients: ricotta cheese, mozzarella cheese, truffl

In [12]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate interesting dish names and descriptions for the 5 menu items based on Italian food choices.")
{'Agent Output': 'The agent named Creative Writer has successfully completed the task of generating interesting dish names and descriptions for 5 menu items based on Italian food choices. The dishes created are as follows:\n1. Bella Vita Pasta: A delightful combination of al dente spaghetti tossed in a rich tomato and basil sauce, topped with freshly grated Parmesan cheese. The perfect representation of the Italian zest for life.\n2. Tuscan Sunset R

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [13]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [14]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [15]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the Assigned Task.
Thoughts: To complete the Assigned Task, I need to generate 5 Italian dish names and descriptions. Since no specific requirements have been provided, I will need to rely on a general function to generate these names and descriptions.
Subtask identified: Generate 5 Italian dish names and descriptions using the use_llm function.
Getting LLM to perform the following task: Generate 5 Italian dish names and descriptions using the use_llm function.
> 1. Bella Vita Pasta: A delightful combination of al dente pasta tossed in a rich tomato and basil sauce, topped with freshly grated Parmesan cheese. This dish will transport you to the charming streets of Italy with every bite.
2. Tuscan Sunrise Risotto: Creamy Arborio rice cooked to perfection with sun-dried t

In [16]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': 'The agent named Creative Writer has successfully completed the task by generating 5 Italian dish names and descriptions. The dishes include Bella Vita Pasta, Tuscan Sunrise Risotto, Amore Alla Carbonara, Sicilian Citrus Seafood Medley, and Venetian Dream Tiramisu. Each dish has a unique description highlighting the flavors and essence of Italian cuisine.'}

Is Task Completed: False


In [17]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: No subtasks have been completed for the Assigned Task. The dish names and descriptions have been generated by the agent named Creative Writer.
Thoughts: Since no subtasks have been completed, the first step is to identify the ingredients for each of the 5 Italian dishes. The agent named Chef should focus on extracting the ingredients based on the dish names provided.
Subtask identified: Extract the ingredients for the 5 Italian dishes: Bella Vita Pasta, Tuscan Sunrise Risotto, Amore Alla Carbonara, Sicilian Citrus Seafood Medley, and Venetian Dream Tiramisu.
Getting LLM to perform the following task: Extract the ingredients for the 5 Italian dishes: Bella Vita Pasta, Tuscan Sunrise Risotto, Amore Alla Carbonara, Sicilian Citrus Seafood Medley, and Venetian Dream Tiramisu.
> For the 5 Italian dishes:
1. Bella Vita Pasta - Ingredients: Pa

In [18]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: No subtasks have been completed yet for the Assigned Task. The dish names and their descriptions have been generated by the Creative Writer, and the ingredients for the dishes have been provided by the Chef.
Thoughts: Since no subtasks have been completed yet, the first step is to calculate the prices for the 5 Italian dishes. This can be achieved by using the dish_price function.
Subtask identified: Use the dish_price function to calculate the prices for the 5 Italian dishes: Bella Vita Pasta, Tuscan Sunrise Risotto, Amore Alla Carbonara, Sicilian Citrus Seafood Medley, and Venetian Dream Tiramisu.
Calling function dish_price with parameters {'list_of_dish_names': ['Bella Vita Pasta', 'Tuscan Sunrise Risotto', 'Amore Alla Carbonara', 'Sicilian Citrus Seafood Medley', 'Venetian Dream Tiramisu']}
> {'output_1': {'Bella Vita Pasta': 

In [19]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: The agent named Creative Writer has generated 5 Italian dish names and descriptions, the Chef has provided the ingredients for the 5 Italian dishes, and the Economist has generated prices for the 5 Italian dishes.
Thoughts: To complete the remainder of the Assigned Task, we need to select only 3 Italian dishes that will please Gordon Ramsay. Since this task involves decision-making based on specific criteria, the Equipped Function 'use_llm' will not be suitable. We will need to analyze the descriptions, ingredients, and prices of the dishes to make the best selection.
Subtask identified: End Task
Task completed successfully!

###
Reply from Boss to Menu Creator:
As the agent named Boss, I have reviewed the information provided in the Global Context and the completed subtasks. From the list of 5 Italian dishes gene

In [20]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

1. Bella Vita Pasta, Bella Vita Pasta is a delightful pasta dish that combines the flavors of cherry tomatoes, basil, and Parmesan cheese to create a truly authentic Italian experience. Ingredients: Pasta, Olive Oil, Garlic, Cherry Tomatoes, Basil, Parmesan Cheese, Price: $10
2. Amore Alla Carbonara, Amore Alla Carbonara is a classic Italian dish featuring spaghetti, pancetta, eggs, and Parmesan cheese, seasoned with black pepper for a rich and creamy flavor. Ingredients: Spaghetti, Pancetta, Eggs, Parmesan Cheese, Black Pepper, Price: $9
3. Venetian Dream Tiramisu, Venetian Dream Tiramisu is a heavenly dessert made with layers of ladyfingers soaked in espresso and creamy mascarpone cheese, dusted with cocoa powder for a perfect finish. Ingredients: Ladyfingers, Espresso, Mascarpone Cheese, Eggs, Sugar, Cocoa Powder, Price: $1


In [21]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': 'The agent named Creative Writer has successfully completed the task by generating 5 Italian dish names and descriptions. The dishes include Bella Vita Pasta, Tuscan Sunrise Risotto, Amore Alla Carbonara, Sicilian Citrus Seafood Medley, and Venetian Dream Tiramisu. Each dish has a unique description highlighting the flavors and essence of Italian cuisine.'}

Subtask: Chef(instruction="Generate ingredients for the 5 Italian dishes")
{'Agent Output': 'For the 5 Italian dishes:\n1. B

In [22]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

The 3 Italian dishes selected to serve Gordon Ramsay are as follows:

1. Bella Vita Pasta
Description: Bella Vita Pasta is a delightful pasta dish that combines the flavors of cherry tomatoes, basil, and Parmesan cheese to create a truly authentic Italian experience.
Ingredients: Pasta, Olive Oil, Garlic, Cherry Tomatoes, Basil, Parmesan Cheese
Price: $10

2. Amore Alla Carbonara
Description: Amore Alla Carbonara is a classic Italian dish featuring spaghetti, pancetta, eggs, and Parmesan cheese, seasoned with black pepper for a rich and creamy flavor.
Ingredients: Spaghetti, Pancetta, Eggs, Parmesan Cheese, Black Pepper
Price: $9

3. Venetian Dream Tiramisu
Description: Venetian Dream Tiramisu is a heavenly dessert made with layers of ladyfingers soaked in espresso and creamy mascarpone cheese, dusted with cocoa powder for a perfect finish.
Ingredients: Ladyfingers, Espresso, Mascarpone Cheese, Eggs, Sugar, Cocoa Powder
Price: $1


In [23]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
The 3 Italian dishes selected to serve Gordon Ramsay are as follows:

1. Bella Vita Pasta
Description: Bella Vita Pasta is a delightful pasta dish that combines the flavors of cherry tomatoes, basil, and Parmesan cheese to create a truly authentic Italian experience.
Ingredients: Pasta, Olive Oil, Garlic, Cherry Tomatoes, Basil, Parmesan Cheese
Price: $10

2. Amore Alla Carbonara
Description: Amore Alla Carbonara is a classic Italian dish featuring spaghetti, pancetta, eggs, and Parmesan cheese,

In [24]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", 'The 3 Italian dishes selected to serve Gordon Ramsay are as follows:\n\n1. Bella Vita Pasta\nDescription: Bella Vita Pasta is a delightful pasta dish that combines the flavors of cherry tomatoes, basil, and Parmesan cheese to create a truly authentic Italian experience.\nIngredients: Pasta, Olive Oil, Garlic, Cherry Tomatoes, Basil, Parmesan Cheese\nPrice: $10\n\n2. Amore Alla Carbonara\nDescription: Amore Alla Carbonara is a classic Italian dish featuring spaghetti, pancetta, eggs, and Parmesan cheese, seasoned with black pepper for a rich and creamy flavor.\nIngredients: Spaghetti, Pancetta, Eggs, Parmesan Cheese, Black Pepper\nPrice: $9\n\n3. Venetian Dream Tiramisu\nDescription: Venetian Dream Tiramisu is a heavenly dessert made with layers of ladyfingers soaked in espresso and creamy mascarpone cheese, dusted with cocoa powder for a perfect finish

In [25]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False
